In [1]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

"""
        i.dcn,
        i.seqnum,
        i.sectitle,
        i.personID,
        i.rolecode1,
        i.rolecode2,
        i.rolecode3,
        i.rolecode4,
        i.tprice,
        i.tprice_adj,
        i.tprice_ar,
        i.trancode,
        i.trancode_ar,
        i.shares,
        i.shares_adj,
        i.sigdate,
        i.secdate,
        i.ticker,
        i.acqdisp,
        i.optionsell,
        i.cleanse,
        i.trandate,
        i.trandate_ar,
                c_next.iid = c.iid AND

"""
# Establish a connection to the WRDS database
db = wrds.Connection(wrds_username='asherbaraban')

columns = db.get_table(library='tr_insiders', table='table1', obs=1).columns
columns_list = ', '.join(f'i.{col}' for col in columns)


df = db.raw_sql(f"""
    SELECT
        i.*,
        c.prccd AS trandate_price,
        c.datadate AS trandate_datadate,
        AVG(c_next.prccd) AS avg_prccd_6mo,
        c.cusip,
        c.iid,
        c.tic
    FROM
        (
            SELECT *,
            CONCAT(cusip6, cusip2, cusipx) AS cusipI
            FROM tr_insiders.table1
            WHERE 
                formtype = '4' AND
                cleanse IN ('R', 'H') AND
                trancode IS NOT NULL AND
                acqdisp IS NOT NULL
        ) AS i
    LEFT JOIN
        comp_na_daily_all.secd AS c
    ON 
        i.cusipI = c.cusip AND
        c.datadate = i.trandate
    LEFT JOIN
        comp_na_daily_all.secd AS c_next
    ON
        i.cusipI = c.cusip AND
        c_next.datadate BETWEEN i.trandate + INTERVAL '6 months' AND i.trandate + INTERVAL '6 months' + INTERVAL '4 days'
    GROUP BY
        {columns_list}, i.cusipI, c.prccd, c.datadate, c.cusip, c.iid, c.tic
""")

filtered_df = df.groupby(['dcn', 'seqnum']).filter(lambda x: x['iid'].nunique() > 1)

df.to_csv('raw_trades.csv', index=False)

Loading library list...
Done


OperationalError: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
    SELECT
        i.*,
        c.prccd AS trandate_price,
        c.datadate AS trandate_datadate,
        AVG(c_next.prccd) AS avg_prccd_6mo,
        c.cusip,
        c.iid,
        c.tic
    FROM
        (
            SELECT *,
            CONCAT(cusip6, cusip2, cusipx) AS cusipI
            FROM tr_insiders.table1
            WHERE 
                formtype = '4' AND
                cleanse IN ('R', 'H') AND
                trancode IS NOT NULL AND
                acqdisp IS NOT NULL
        ) AS i
    LEFT JOIN
        comp_na_daily_all.secd AS c
    ON 
        i.cusipI = c.cusip AND
        c.datadate = i.trandate
    LEFT JOIN
        comp_na_daily_all.secd AS c_next
    ON
        i.cusipI = c.cusip AND
        c_next.datadate BETWEEN i.trandate + INTERVAL '6 months' AND i.trandate + INTERVAL '6 months' + INTERVAL '4 days'
    GRO

In [ ]:
# Step 2: Calculate the absolute difference between 'tprice_adj' and 'trandate_price'
filtered_df['price_diff'] = (filtered_df['tprice_adj'] - filtered_df['trandate_price']).abs()

# Step 3: Sort by 'dcn', 'seqnum', and the absolute difference in ascending order (default)
sorted_df = filtered_df.sort_values(by=['dcn', 'seqnum', 'price_diff'])

# Step 4: Group by 'dcn' and 'seqnum' and keep the first row for each group (closest 'tprice_adj' to 'trandate_price')
deduplicated_df = sorted_df.groupby(['dcn', 'seqnum']).first().reset_index()

# Step 5: Identify rows to drop from the original DataFrame (those in filtered_df)
rows_to_drop = filtered_df.index

# Step 6: Drop the identified rows from the original DataFrame
df_remaining = df.drop(rows_to_drop)

# Step 7: Concatenate the deduplicated rows back into the remaining DataFrame
final_df = pd.concat([df_remaining, deduplicated_df], ignore_index=True)

final_df.head()

# final_df.to_csv('insider_trading_raw.csv', index=False)